# Status 

* Parsing Input ( OK )
* Data Analysis, find opportunities (OK)
* first optimization algorithm : orders for warehouses (OK)
* second optimization algorithmn : Routing (OK) 
* Postprocessing and output writing (In progress)
* Write a conclusion


# Libraries

* **Pandas** : To handle data
* **PuLP** : Linear programing library. it is used to solve the assignment problem (order fo warehouse)
* **Google or-tools** : constraint programming library. It is used to solve the routing problem  
* **PyPlot** : Plotting graph


# Imports

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Chart drawing init
import plotly as py
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(255,255,2555,0.5)', plot_bgcolor='rgba(250,250,250,0.8)', autosize=True)#, width=1200, height=800)
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'


import plotly.io as pio

# Input Data

## Parsing 

In [ ]:
distance = lambda x1, y1, x2, y2:np.sqrt(np.abs(x1-x2)**2 + np.abs(y1-y2)**2)

In [ ]:
files = ['../input/hashcode-drone-delivery/busy_day.in']


file_name = files[0]
with open(file_name) as file:
    lines = file.read().splitlines()
line_list = [[int(n) for  n in ll.split()] for ll in lines]

#line_list

In [ ]:
# Params
R = line_list[0][0]
C = line_list[0][1]
n_drones = line_list[0][2]
T = line_list[0][3]
payload = line_list[0][4]

print("Periods : ", T)
print("Payload : ", payload)
print("Drones : ", n_drones)
print("Grid : (", R,",", C, ')')

# Products
n_products = line_list[1][0]
product_weight = line_list[2]
assert n_products == len(product_weight)

print("Product types : ", n_products)

# Warehouses
n_warehouses = line_list[3][0]

warehouses = []
wh_stocks = []

print("There is ", n_warehouses, " warehouses for ", T, " rounds")

for i in np.arange(4, 2*(n_warehouses) + 4, 2):
    warehouses.append( (line_list[i][0], line_list[i][1]) )
    wh_stocks.append( line_list[i+1] )
    end_wh = i+2
    
# print("end ", end_wh)

assert len(warehouses) == n_warehouses
assert len(warehouses) == len(wh_stocks)

In [ ]:
# Orders
order_startline = 2*(n_warehouses+1)+2 # last list of wh + 2 line 
print(end_wh, order_startline)
assert order_startline == end_wh

n_order = line_list[end_wh][0]
print("There is : ", n_order, " orders")

order_delivery = []
order_n_items = []
order_quantity_items = []

print(3*(n_order) + (order_startline+1))

end_order = 3*(n_order) + (order_startline+1)

for i in np.arange(order_startline+1, 3*(n_order) + (order_startline+1), 3):  
    order_delivery.append(line_list[i])
    order_n_items.append(line_list[i+1][0])
    order_quantity_items.append(line_list[i+2])
    end_o = i+3
    

print(len(order_quantity_items))
assert n_order == len(order_n_items)
assert n_order == len(order_delivery)
assert n_order == len(order_quantity_items)

assert order_n_items[32] == len(order_quantity_items[32])

In [ ]:
print("Information : ")
print("Grid : (", R, ",", C, ")")
print("Drones : ", n_drones, " with payload ", payload)
print("Warehouses : ", n_warehouses)
print("orders : ", n_order)

## Aggregation

In [ ]:
# 2 dataframes : orders and warehouses

In [ ]:
d_wh = {
    "warehouse" : [i for i in range(0, n_warehouses)],
    "x" : [i[0] for i in warehouses],
    "y" : [i[1] for i in warehouses],
}


# init
for i in range(n_products):
    p_id = 'prod_'+str(i)
    d_wh[p_id] = []

for j in range(n_warehouses):
    for i in range(n_products):
        p_id = 'prod_'+str(i)
        d_wh[p_id].append(wh_stocks[j][i])

df_wh = pd.DataFrame(d_wh)

df_wh.head()

In [ ]:
d_order = {
    "order" :        [i for i in range(0, n_order)],
    "items" :        order_n_items,
    "total_weight" : [0]*n_order,
    "x":             [order[0] for order in order_delivery],
    "y" :            [order[1] for order in order_delivery],
}

# init
for i in range(n_products):
    p_id = 'prod_'+str(i)
    d_order[p_id] = [0]*n_order

for j in range(0, n_warehouses):
    d_order["wh_"+str(j)] = [0]*n_order
    d_order["dist_"+str(j)] = [0]*n_order

    
for k in range(n_order): 
    w = 0
    for i in order_quantity_items[k]:
        p_id = 'prod_'+str(i)
        d_order[p_id][k] += 1
        w += product_weight[i]
           
        
        for j in range(0, n_warehouses):
            if df_wh[p_id][j] > 0:
                d_order["wh_"+str(j)][k] += 1
            else:
                d_order["wh_"+str(j)][k] += 0
            
    
        # nearest wh
        for j in range(0, n_warehouses):
            d_order["dist_"+str(j)][k] = distance(d_order['x'][k], d_order['y'][k],
                                                 df_wh['x'][j], df_wh['y'][j])
            
    
    d_order["total_weight"][k] += w
    


df_order = pd.DataFrame(d_order)
df_order['nearest_wh'] = df_order[['dist_'+str(i) for i in range(0, n_warehouses)]].T.idxmin().apply(lambda x: x.split('_')[1])

df_order.head()

In [ ]:
# df_order = df_order.query('items == 1').reset_index()
# n_order = df_order.shape[0]

#### Test

In [ ]:
n = 1150
order_quantity_items[n].sort()
order_quantity_items[n]

for p in order_quantity_items[n]:
    assert int(df_order[df_order.order == n]['prod_'+str(p)]) > 0

# Analysis

In [ ]:
df_order.describe()

## Product weight 

In [ ]:
pd.Series(product_weight).plot.box()

## Map : Orders and warahouses in Grid

In [ ]:
import plotly.express as px

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Scatter(x=df_wh['x'], y=df_wh['y'],
                         name='warehouses',marker_size=15,
                         marker_color='red', mode='markers', text=df_wh['warehouse']), row=1, col=1)



fig.add_trace(go.Scatter(x=df_order['x'], y=df_order['y'],
                         name='orders',marker_size=3,
                         marker_color='blue', mode='markers'), row=1, col=1)


fig.show()

## Nearest warehouse distribution

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Histogram(x=df_order['nearest_wh'], name='Nearest warehouse distribution', marker_color="purple"), row=1, col=1)


fig.show()

**=> most of order are close to warehouse 4 (330) and 3 (267)**

## Product availability distribution by warehouses

In [ ]:
df_wh.T.sample(5)

In [ ]:
# product availability by warehouse

fig = make_subplots(rows=10, cols=2)

for i in range(0, 10):
    col = 1 if i%2 == 0 else 2
    row = i+1 if i%2 == 0 else i
    fig.add_trace(go.Histogram(x=df_wh.T[i], name='wh'+str(i)), row=row, col=col)

fig.show()

Some products such as 399 or 395 are available in wh_5 and wh_6. this information is important whether an heuristic is create 

## Order distribution : Order weight and item per Order

In [ ]:
df_order.sample(5)

In [ ]:
fig = make_subplots(rows=2, cols=2)


fig.add_trace(go.Histogram(x=df_order.total_weight, name='order weight', marker_color="blue"), row=1, col=1)


fig.add_trace(go.Histogram(x=df_order['items'], name='items per order', marker_color="purple"), row=1, col=2)


for i in range(1, 7):
    fig.add_trace(go.Scatter(x=[i*payload, i*payload], y=[0, 100], name=str(i)+"x payload", marker_color='red'), row=1, col=1)


fig.add_trace(go.Histogram(x=product_weight, name='product weight', marker_color="green"),
        row=2, col=1)
    
fig.show()

## Order and warehouses

### How many warehouses are needed to supply the orders ?

In [ ]:
%%time
for i in range(0, 10):
    print(df_order[['order', 'items', 'wh_'+str(i), 'dist_'+str(i), 'nearest_wh']].query("nearest_wh == "+str(i)+" and wh_"+str(i)+" == items").shape)

**=> nobody could be totaly supplied by its nearest warehouse**

In [ ]:
%%time

count = 0
for i in range(0, 10):
    n = df_order[['order', 'items', 'wh_'+str(i), 'dist_'+str(i), 'nearest_wh']].query("wh_"+str(i)+" == items").shape[0]
    print('Number of order which could be supplid by warehouse '+str(i)+' only: ',n,' / '+str(n_order))
    count += n


print("\nBy assigning orders to warehouses we could supply ", count, ' / '+str(n_order), '\n')

### => A strategy could be to handle these 977 orders only

# Linear Programming

## 3 Steps 

1. Assign order to warehouse based on warehouse stocks
2. Assign a fleet of drone for each warehouse ( example : 10 drones per warehouse)
3. foreach warehouse apply a Routing algorithm (CVRP with capacity constraint)

## 1.Assign orders to warehouses

To solve this subProblem we're going to PuLP to solve the Linear Program

In [ ]:
! pip install pulp

from pulp import * 

In [ ]:
df_order.sample(5)

## 1. From each warehouse, assign product to order based on demand

### Problem definition:
* i is order index $i \in Orders$
* j is warehouse index $ j \in warehouses$
* k is product index $ k \in products$
* $dist_{ij}$ is the distance between the order i and the warehouse j
* $demand_{ki}$ defines the demand of product k from the order i (basically 1 or 0 in the case)
* $x_{ij}^k$ quantity of product k for the order i from the warehouse j



min $\sum\limits_{k,i,j} x_{ij}^k . dist_{ij}$

Subject to 

1. $\sum\limits_{j \in warehouses}  \sum\limits_{i \in Orders} demand_{k,i} . x_{ij}^k \leq \sum\limits_{j \in warehouses} Stock_{j, k}$  $\forall k \in products $ *(Respect of stock)*
2. $\sum\limits_{j \in warehouses}  x_{ij}^k = demand_{k,i}$ $\forall i \in orders$  *(Every order have at leat one warehouse)*

In [ ]:
# %%time


# Stock of item k in warehouse j
stock = [[0]*n_products]*n_warehouses
for wh in range(n_warehouses):
    for prod in range(n_products):
        stock[wh][prod] = wh_stocks[wh][prod]
stock = np.array(stock)


# Stock of item k in warehouse j
# cols = ['prod_'+str(i) for i in range(n_products)]
# stock = np.array(df_wh[cols]).astype('int32')
# assert stock.shape == (n_warehouses, n_products)
# assert stock[0][10] == 4
# assert stock[2][7] == 36


# Number of item k in order i
# demand = np.array(df_order[cols].T)
# assert demand.shape == (n_products, n_order)
# assert demand[6][0] == 1
# assert demand[18][4] == 1


# Number of item k in order i
demand = np.array([[0]*n_order]*n_products)
for order in range(n_order):
    for prod in order_quantity_items[order]:
        demand[prod][order] += 1
demand = np.array(demand)



# min(stock[k][j],
# Variable product k from warehouse j is for order i
x = [[[LpVariable("x_"+str(k)+"_"+str(i)+"_"+str(j), 0,  min(1, demand[k][i]), LpInteger) 
       for j in range(n_warehouses)] 
      for i in range(n_order) ] 
     for k in range(n_products) ] 




# distance between order i and warehouse j 
cols0 = ['dist_'+str(j) for j in range(n_warehouses)]
dist = np.array(df_order[cols0]).astype('float32')

In [ ]:
stock = [[0]*n_products]*n_warehouses
for wh in range(n_warehouses):
    for prod in range(n_products):
        stock[wh][prod] += wh_stocks[wh][prod]

stock = np.array(stock)

In [ ]:
%%time 

# Problem
prob = LpProblem("assign_product_to_order", LpMinimize)


prob += lpSum(dist[i][j] * x[k][i][j] for k in range(n_products) for i in range(n_order) for j in range(n_warehouses)), "obj"


# Order Availability            
for k in range(n_products): 
    prob += lpSum(x[k][i][j] for i in range(n_order) for j in range(n_warehouses) ) <= lpSum(stock[j][k] for j in range(n_warehouses)), "Stock cst"+str(k)#+"_"+str(j)


for i in range(n_order): 
    for k in range(n_products):
        prob += lpSum( x[k][i][j] for j in range(n_warehouses) ) == demand[k][i], "demand cst"+str(k)+"_"+str(i)



# Write the problem as an LP file
prob.writeLP("ass_order_item_prob.lp")

print('ok')

In [ ]:
%%time


# Solve the problem using the default solver
prob.solve()

# Print the status of the solved LP
print("Status:", LpStatus[prob.status])

# Print the value of the variables at the optimum
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# Print the value of the objective
print("objective=", value(prob.objective))

## LP Postprocessing 

In [ ]:
%%time


z = np.array([[0]*n_warehouses]*n_order).astype('int32')
df_assign = pd.DataFrame(columns = ['warehouse', 'order', 'product_id', 'quantity', 'total_weight'])


for k in range(n_products):
    for i in range(n_order):
        for j in range(n_warehouses):
            if x[k][i][j].varValue > 0:
                z[i][j] = 1
                
                df_assign = df_assign.append({
                    'warehouse':    j,
                    'order':        i,
                    'product_id':   k,
                    'quantity' :    int(x[k][i][j].varValue), 
                    'total_weight': int(x[k][i][j].varValue) * product_weight[k]
                }, ignore_index=True)
                
                
print(df_assign.shape)



# little check for post processing

order = 10
tmp = df_assign.sort_values('warehouse').query('order == '+str(order))
prods = tmp.product_id

for p in order_quantity_items[10]:
    assert np.isin(p, prods)


# df_assign.head()

In [ ]:
df_assign.sort_values('warehouse').query('quantity > 2').head()

In [ ]:
%%time

for i in range(n_order):
    if sum(z[i]) > 1:
        print("more than 1 wh ",i, ", sum =  ", sum(z[i]))
    if sum(z[i]) == 0:
        print("0 wh", i)
     
    
for j in range(n_warehouses):
    df_order['assign_'+str(j)] = z.T[j]

In [ ]:
df_order[['order', 'items']+['assign_'+str(j) for j in range(n_warehouses) ]].sample(5)

## Result in graph

In [ ]:
import plotly.express as px

fig = make_subplots(rows=1, cols=1)


ref_color = ['red','blue',  'brown', 'purple', 'orange', 'cyan', 'black', 'yellow', 'pink', 'green']

for i in range(n_warehouses):
    tmp = df_wh.iloc[i]
    color = ref_color[i]
    fig.add_trace(go.Scatter(x=[tmp['x']], y=[tmp['y']],
                             name='warehouse '+str(i),
                             marker_size=15,
                             marker_color=color,
                             mode='markers', 
                            text='warehouse '+str(i)
                            ), row=1, col=1)


    tmp_ord = df_order.query("assign_"+str(i)+" >= 1.0")
    print(tmp_ord.shape[0], " orders for warehouse "+str(i), ' colorized in ', color)
    
    fig.add_trace(go.Scatter(x=tmp_ord['x'], y=tmp_ord['y'],
                            name='orders for warehouse '+str(i),
                            marker_size=3,
                            marker_color=color,
                            mode='markers'), row=1, col=1)

fig.show()

### => The clustering linear program seems to be good 

In [ ]:
df_assign.sample(5, random_state=89)

# Routing heuristic

## 3. Assign drone to Order and schedule delivery

Now we know which warehouse has to handle which order.

To simplify, we will give **10 drones per warehouses**. ( for sure, we could create a repartition based on the total weight to manage per warehouse).

For each, warehouse a routing algorithm will be applied (**CVRP** with capacity. a time window constraint based on the distance could be applied to improve the score).


Fortunately, it exists very good routing problem solver such as the one in **OR-Tools**. we'll use it. https://developers.google.com/optimization/routing

### Let's start with warehouse 0 and 10 drones

In [ ]:
warehouse_order = []
wh_order_dist = []

for j in range(n_warehouses):
    tmp = df_assign.query("warehouse == "+str(j))
    warehouse_order.append(tmp)
    print('Warehouse ', j,' : ', tmp.shape[0], 'products to deliver')
    
    #distances
    wh_order_dist.append([0] + df_order['dist_'+str(j)].tolist())
    

## CVRP : Example with warehouse 0

In [ ]:
wh = 1
n_drone_per_wh = n_drones//10

df = df_assign.query("warehouse == "+str(wh)).sort_values('order')

order_list = list(df.order) # -1 for the warehouse (index = 0)
print(len(order_list))

df.head()

In [ ]:
%%time

# demands : weight of each product ordered 
demands = df['total_weight'].tolist()+ [0]
products = df['product_id'].tolist()+[0]
len(demands )


# define distances 
X = np.array(d_order['x'])   
Y = np.array(d_order['y'])
         
ref_dist = np.array([[ distance(X[i], Y[i], X[j], Y[j]) for j in range(n_order)] for i in range(n_order)])



tmp = pd.DataFrame(np.array([[ref_dist[i][j] for j in order_list] for i in order_list]))


# last row and first column: distance with the warehouse 0

tmp[tmp.shape[0]] = pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0])
tmp = tmp.append(pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0]),  ignore_index=True)
distance_ = np.array(tmp)

print(distance_.shape)

distance_

In [ ]:
print(distance_.shape)
print(len(demands))

pd.DataFrame(distance_).tail(5)

In [ ]:
turns = 30 # nb travels per drones
n_drone_per_wh = 10 * turns

In [ ]:
turns = [30, 30, 30, 100, 100, 100, 40, 40, 30]


def create_data_model(wh=0):
    """Stores the data for the problem."""
    
   #  wh = 1
    n_drone_per_wh = n_drones//10
    df = df_assign.query("warehouse == "+str(wh)).sort_values('order')
    order_list = list(df.order)
   
    # demands : weight of each product ordered 
    demands = df['total_weight'].tolist()+[0]
    products = df['product_id'].tolist()+[0]
    len(demands )


    # define distances 
    X = np.array(d_order['x'])   
    Y = np.array(d_order['y'])

    ref_dist = np.array([[ distance(X[i], Y[i], X[j], Y[j]) for j in range(n_order)] for i in range(n_order)])
    tmp = pd.DataFrame(np.array([[ref_dist[i][j] for j in order_list] for i in order_list]))


    # last row and first column: distance with the warehouse 0
    tmp[tmp.shape[0]] = pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0])
    tmp = tmp.append(pd.Series([df_order.iloc[i]['dist_'+str(wh)] for i in order_list]+[0]),  ignore_index=True)
    distance_ = np.array(tmp)

    # print(distance_.shape)
    turns = [30, 30, 30, 2000, 400, 100, 100, 100, 100,100]
    turns = turns[wh] # nb travels per drones
    n_drone_per_wh = 10 * turns
    
    
    data = {}
    data['distance_matrix'] = distance_
    data['demands'] = demands
    data['vehicle_capacities'] = [payload]*n_drone_per_wh
    data['num_vehicles'] = n_drone_per_wh
    data['depot'] = tmp.shape[0]-1 # last one is the warehouse
    
    data['products'] = products
    data['order_list'] = order_list+[0]
    
    # not used yet
    data['time_matrix'] = distance_
    data['vehicle_load_time'] = 1
    data['vehicle_unload_time'] = 1
    data['time_windows'] = [(0, T)]*len(demands)
    data['depot_capacity'] = n_drone_per_wh

    return data

In [ ]:
%%time 
a = create_data_model(3)

In [ ]:
# """Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(len(dropped_nodes.split(' ')))
    print(dropped_nodes)
   
    
    total_distance = 0
    total_load = 0
    count = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        
        
        if route_load > 0:
            print(plan_output)
            count += 1
        
        total_distance += route_distance
        total_load += route_load
    print('Total needed rounds : {} '.format(count//10))
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))

In [ ]:
def main(wh=0, print_solution=False):
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(wh)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    
    
    # Add Time Windows constraint.
#     time = 'Time'
#     routing.AddDimension(
#         transit_callback_index,
#         30,  # allow waiting time
#         30,  # maximum time per vehicle
#         False,  # Don't force start cumul to zero.
#         time)
#     time_dimension = routing.GetDimensionOrDie(time)
#     # Add time window constraints for each location except depot.
#     for location_idx, time_window in enumerate(data['time_windows']):
#         if location_idx == data['depot']:
#             continue
#         index = manager.NodeToIndex(location_idx)
#         time_dimension.CumulVar(index).SetRange(int(time_window[0]), int(time_window[1]))
#     # Add time window constraints for each vehicle start node.
#     for vehicle_id in range(data['num_vehicles']):
#         index = routing.Start(vehicle_id)
#         time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0], data['time_windows'][0][1])
                        
    
                                                
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Allow to drop nodes.
    penalty = 15000
    for node in range(1, len(data['distance_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

    
    
    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    # search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.AUTOMATIC)
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    # search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GREEDY_DESCENT)
    search_parameters.log_search = True
    search_parameters.time_limit.FromSeconds(1200) # 15 minutes 


    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    print("Solver status: ", routing.status())
    # Print solution on console.
    # solution = True
    if print_solution:
        if solution:
            print_solution(data, manager, routing, solution)
    
    return (data, manager, routing, solution)

In [ ]:
%%time

# (data, manager, routing, solution) = main(3)

In [ ]:
def main_df(wh=0):
    (data, manager, routing, solution) = main(wh, False)
    
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(len(dropped_nodes.split(' ')))
    print(dropped_nodes)


    # Display dropped nodes.
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node:
            dropped_nodes += ' {}'.format(manager.IndexToNode(node))
    print(len(dropped_nodes.split(' ')))
    print(dropped_nodes)

    total_distance = 0
    total_load = 0
    count = 0

    # Solution Dataframe
    df_route = pd.DataFrame(columns=['warehouse', 'route', 'order', 'product', 'load'])

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            prod = data['products'][node_index]

            plan_output += ' {0} Load({1} - product: {2} ) -> '.format(data['order_list'][node_index], route_load, prod)

            tmp_data = {'warehouse': wh, 
                        'route': vehicle_id, 
                        'product': prod, 
                        'order': data['order_list'][node_index], 
                        'load': route_load }
            #print(tmp_data)
            df_route = df_route.append(tmp_data, 
                                       ignore_index=True)
            #print(df_route.head(3))

            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

    
        # Possibility to improve by taking the nearest warehouse
#         prod = data['products'][node_index]
#         plan_output += ' {0} Load({1} - product: {2} ) \n '.format(node_index, route_load, prod)
#         df_route = df_route.append({'warehouse': 0, 'route': vehicle_id, 
#                                     'product': prod, 
#                                     'order': data['order_list'][node_index], #node_index, 
#                                     'load': route_load }, ignore_index=True)

        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)

        if route_load > 0:
            print(plan_output)
            count += 1
        total_distance += route_distance
        total_load += route_load
        
    print('Total needed rounds : {} '.format(count//10))
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    
    return df_route

In [ ]:
%%time

#df_route = main_df(3)

In [ ]:
%%time 

warehouses = [0, 1, 2, 5, 6, 7, 8, 9 ]


tmp_list = []
df_route = pd.DataFrame()
for wh in warehouses:# range(n_warehouses):
    print("\n\nWarehouse : ", wh)
    tmp = main_df(wh)
    tmp_list.append(tmp)

In [ ]:
df_route = pd.concat(tmp_list)

# test 
order = 121
tmp = df_route.query('order == '+str(order))

for prod in order_quantity_items[order]:
    assert np.isin(prod, tmp['product'])
    

df_route.shape

In [ ]:
df_route = df_route.query('load != 0')
print("instruction for warehouse 1 : ", df_route.query("warehouse == "+str(1)).shape[0] )

# Solution in graph

In [ ]:
fig = make_subplots(rows=1, cols=1)

ref_color = ['red','blue',  'brown', 'yellow', 'cyan', 'orange', 'black', 'purple', 'pink', 'green']


for wh in warehouses: #range(n_warehouses):
    # wh = 0
    tmp = df_wh.iloc[wh]
    color = ref_color[wh]
    fig.add_trace(go.Scatter(x=[tmp['x']], y=[tmp['y']],
                             name='warehouse '+str(wh),
                             marker_size=15,
                             marker_color=color,
                             mode='markers', 
                            text='warehouse '+str(wh)
                            ), row=1, col=1)


    tmp_wh = df_wh.iloc[wh]

    # print("Order : ", order)
    
    orders = list(df_route.query("warehouse == "+str(wh)).order.unique())
    order_x = df_order.query("order in "+str(orders))['x']
    order_y = df_order.query("order in "+str(orders))['y']

    #print(tmp_ord.shape[0], " orders for warehouse "+str(order), ' colorized in ', color)

    fig.add_trace(go.Scatter(x=order_x, y=order_y,
                            name='orders for warehouse '+str(wh),
                            marker_size=6,
                            marker_color=color,
                            mode='markers'), row=1, col=1)


    # line
    for r in df_route.query("warehouse == "+str(wh)).route.unique()[:60]: 
        route = df_route.query('warehouse == '+str(wh)+' and route == '+str(r))
        pre_node = (tmp_wh['x'], tmp_wh['y'])
        for i in range(route.shape[0]):
            if i != 0:
                order = route.iloc[i]['order']

                x = df_order['x'][order]
                y = df_order['y'][order]
                fig.add_trace(go.Scatter(x=(pre_node[0], x) , y=(pre_node[1], y), name="route "+str(r),marker_size=1, marker_color=color), row=1, col=1)
                pre_node = (x, y)

        fig.add_trace(go.Scatter(x=[pre_node[0], int(tmp_wh['x'])  ] , y=[pre_node[1], tmp_wh['y'] ], name="route "+str(r),marker_size=1, marker_color=color), row=1, col=1)


fig.show()

# Submission

In [ ]:
df_route['qty'] = [1]*df_route.shape[0]
df_route.groupby(['warehouse', 'route', 'product'])['qty'].sum().reset_index().query('qty > 1 and product != 0')

In [ ]:
%%time

def submit(df_route):
    instruction = ""

    #wh = 1
    time = [T]*n_drones
    fleet_per_wh = n_drones//n_warehouses

    instruction = []
    for wh in warehouses : #range(2):
        print("warehouse : ", wh)
        routes = set(df_route.query('warehouse == '+str(wh)).route)

        # time[drone] -= distance(df_wh['x'][0], df_wh['y'][0], 
        #                        df_wh['x'][wh], df_wh['y'][wh] ) # starting time, from wh to assigned wh
        
        for turn in range(len(routes)):
            instruction_d = [] # delivery
            instruction_l = [] # load

            drone = fleet_per_wh*wh + turn%fleet_per_wh  # drone id
            # drone = turn
            # print("Drone : ", drone)

            #r = routes.pop(0)
            r = turn
            route = df_route.query("route == "+str(r)+" and warehouse == "+str(wh))
            prec = (int(df_wh['x'][wh]), int(df_wh['y'][wh]))
            
            for line in range(route.shape[0]):
                if line > 0:
                    # load phase
                    product = route.iloc[line]['product']
                    qty = 1 # to edit to improve
                    order = route.iloc[line]['order']

                    # Loading phase
                    instruction_l.append("{0} L {1} {2} {3} \n".format(drone, wh, product, qty))
                    time[drone] -= 1

                    # delivery phase
                    instruction_d.append("{0} D {1} {2} {3} \n".format(drone, order, product, qty) )
                    time[drone] -= distance(prec[0], prec[1], df_order['x'][order], df_order['y'][order]) 

                    prec = (df_order['x'][order], df_order['y'][order])  

            # back to warehouse, as mentioned, a better tactic would be to take the nearest warehouse
            time[drone] -= distance(prec[0], prec[1], df_wh['x'][wh], df_wh['y'][wh] ) 
            instruction += instruction_l + instruction_d
    print(time)
    return instruction

In [ ]:
instruction = submit(df_route)
print(len(instruction))
instruction[:6]

In [ ]:
# Write file

file1 = open("submission.csv","w") # append mode 
file1.write(str(len(instruction))+'\n')
for line in instruction:
    file1.write(line) 
file1.close() 

# Conclusion 

In [ ]:
order_quantity_items[121]